In [1]:
%cd ..
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from melp import Detector
import melp
import math
import random
from glob import glob

from melp.clustering.misc import*
import melp.clustering as clump
import melp.clustering.time_cluster as tclump
#from melp import TileAnalyzer
import melp.clustering.spatial_cluster as sclump
import melp.clustering.plots as clump_plt
import melp.clustering.three_frame_cluster as clump_3

plt.rcParams.update({'font.size': 16})

/home/erik/GitHub/melp
Welcome to JupyROOT 6.24/06


## Time then iterative spatial clustering

In [2]:
time_threshold_used = 1.2
threshold_cluster_width_used = 999999

function_str = "mt_compare_to_tid"
src = "./testdata/sorted/root_files_fixed/mu3e_sorted_000*.root"

args = (time_threshold_used, threshold_cluster_width_used, "big", None, "timetheniterativespatial")

#clump.multithreading.run_mt(function_str, src, args)

In [5]:
src_res = "./melp/clustering/results"

#src_frac_corr_frame          = src_res + "/frac_corr_frame_tid_*_timetheniterativespatial_{}.txt".format(threshold_cluster_width_used)
#src_frac_corr_clusters_frame = src_res + "/frac_corr_clusters_frame_tid_*_timetheniterativespatial_{}.txt".format(threshold_cluster_width_used)
#src_frac_uncorr_frame        = src_res + "/frac_uncorr_frame_tid_*_timetheniterativespatial_{}.txt".format(threshold_cluster_width_used)
#src_efficiency_stats         = src_res + "/efficiency_stats_tid_*_timetheniterativespatial_{}.txt".format(threshold_cluster_width_used)

src_frac_corr_frame          = src_res + "/frac_corr_frame_tid_*_timetheniterativespatial.txt"
src_frac_corr_clusters_frame = src_res + "/frac_corr_clusters_frame_tid_*_timetheniterativespatial.txt"
src_frac_uncorr_frame        = src_res + "/frac_uncorr_frame_tid_*_timetheniterativespatial.txt"
src_efficiency_stats         = src_res + "/efficiency_stats_tid_*_timetheniterativespatial.txt"

fnames_frac_corr_frame           = glob(src_frac_corr_frame)
fnames_frac_corr_clusters_frame  = glob(src_frac_corr_clusters_frame)
fnames_frac_uncorr_frame         = glob(src_frac_uncorr_frame)
fnames_efficiency_stats          = glob(src_efficiency_stats)

array_frac_corr_frame          = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_frame])
array_frac_corr_clusters_frame = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_clusters_frame])
array_frac_uncorr_frame        = np.concatenate([np.loadtxt(f) for f in fnames_frac_uncorr_frame])
#array_efficiency_stats         = np.concatenate([np.loadtxt(f) for f in fnames_efficiency_stats])

### Efficiency statistics

In [ ]:
for f in fnames_efficiency_stats:
    print("File: " + f)
    with open(f, "r", encoding="utf-8") as file:
        for line in file:
            print(line.strip())
    print("\n")

### Efficiency plots

#### Hits associated to wrong cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_uncorr_frame, bins = 500)
plt.title("Incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of incorrectly associated hits")
plt.savefig("incorr_associated_all_30_files_timetheniterativespatial.png", dpi = 300)
plt.show()

#### Hits associated to correct cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame, bins = 500)
plt.title("Correctly associated hits / all hits in frame (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of correctly associated hits")
plt.savefig("corr_associated_all_30_files_timetheniterativespatial.png", dpi = 300)
plt.show()

#### Combined histograms

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame, bins = 500, label = "Hits correctly associated to cluster / All hits in clusters")
plt.hist(array_frac_uncorr_frame, bins = 500, label = "Hits incorrectly associated to cluster / All hits in clusters")
plt.title("Correctly and incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of total hits")
plt.legend()
plt.savefig("Corr_and_incorr_fractions_all_30_files_timetheniterativespatial.png", dpi = 300)
plt.show()

### Efficiency vs maximum cluster width

In [ ]:
function_str = "mt_efficiency_as_function_of_cluster_width"
cluster_width_array_used = np.arange(10,101,1)
filename_used = "./testdata/sorted/root_files_fixed/mu3e_sorted_000127.root"
args = (filename_used, time_threshold_used, "big", None, None, "timetheniterativespatial")

#clump.multithreading.run_mt(function_str, src, args, cluster_width_array = cluster_width_array_used)

In [ ]:
src_cluster_width    = src_res + "/efficiency_vs_cluster_width_*_timetheniterativespatial.txt"
fnames_cluster_width = glob(src_cluster_width)

array_cluster_width_eff  = []
array_cluster_width      = []
array_cluster_width_tmp  = []
array_cluster_width_tmp.append([np.loadtxt(f) for f in fnames_cluster_width])
for i in array_cluster_width_tmp[0]:
    array_cluster_width_eff.append(float(i[0]))
    array_cluster_width.append(float(i[1]))

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(array_cluster_width, array_cluster_width_eff)
plt.xlabel("Maximum distance between two cluster hits to be included in the efficiency calculation[mm]")
plt.ylabel("Efficiency")
plt.title("Efficiency as function of the maximum distance between clusters with same TID")
plt.savefig("efficiency_vs_max_distance_threshold_10-100_1_all_frames_each_correct_timetheniterativespatial.png", dpi=300)
plt.show()

### Fraction of truth clusters bigger than threshold

In [ ]:
function_str = "mt_frac_truth_cluster_size_filename"
cluster_width_array_used = np.arange(10,101,1)
filename_used = "/home/erik/GitHub/melp/testdata/sorted/root_files_fixed/mu3e_sorted_000127.root"
args = (filename_used, "big", None, None)

#clump.multithreading.run_mt(function_str, src, args, cluster_width_array = cluster_width_array_used)

In [ ]:
src_frac_cluster_over    = src_res + "/frac_truth_Cluster_size_*.txt"
fnames_frac_cluster_over = glob(src_frac_cluster_over)

array_fractions_tmp = []
array_fractions = []
array_thresholds = []
array_fractions_tmp.append([np.loadtxt(f) for f in fnames_frac_cluster_over])
for i in array_fractions_tmp[0]:
    array_fractions.append(float(i[0]))
    array_thresholds.append(float(i[1]))

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(array_thresholds, array_fractions)
plt.xlabel("Threshold for cluster size [mm]")
plt.ylabel("Percentage [%]")
plt.title("Percentage of all truth clusters bigger than threshold")
#plt.savefig("percentage_truth_clusters_over_threshold_all_30_files.png", dpi=300)
plt.show()

## Pure time clustering

In [ ]:
time_threshold_used = 0.4
threshold_cluster_width_used = 20 

function_str = "mt_compare_to_tid"
src_time = "./testdata/sorted/root_files_fixed/mu3e_sorted_000*.root"

args_time = (time_threshold_used, threshold_cluster_width_used, "big", None, "time")

#clump.multithreading.run_mt(function_str, src_time, args_time)

In [ ]:
src_frac_corr_frame_time          = src_res + "/frac_corr_frame_tid_*_time_{}.txt".format(threshold_cluster_width_used)
src_frac_corr_clusters_frame_time = src_res + "/frac_corr_clusters_frame_tid_*_time_{}.txt".format(threshold_cluster_width_used)
src_frac_uncorr_frame_time        = src_res + "/frac_uncorr_frame_tid_*_time_{}.txt".format(threshold_cluster_width_used)
src_efficiency_stats_time         = src_res + "/efficiency_stats_tid_*_time_{}.txt".format(threshold_cluster_width_used)

fnames_frac_corr_frame_time           = glob(src_frac_corr_frame_time)
fnames_frac_corr_clusters_frame_time  = glob(src_frac_corr_clusters_frame_time)
fnames_frac_uncorr_frame_time         = glob(src_frac_uncorr_frame_time)
fnames_efficiency_stats_time          = glob(src_efficiency_stats_time)

array_frac_corr_frame_time          = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_frame_time])
array_frac_corr_clusters_frame_time = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_clusters_frame_time])
array_frac_uncorr_frame_time        = np.concatenate([np.loadtxt(f) for f in fnames_frac_uncorr_frame_time])
#array_efficiency_stats         = np.concatenate([np.loadtxt(f) for f in fnames_efficiency_stats])

### Efficiency statistics

In [ ]:
for f in fnames_efficiency_stats_time:
    print("File: " + f)
    with open(f, "r", encoding="utf-8") as file:
        for line in file:
            print(line.strip())
    print("\n")

### Efficiency plots

#### Hits associated to wrong cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_uncorr_frame_time, bins = 300)
plt.title("Incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of incorrectly associated hits")
#plt.savefig("incorr_associated_all_30_files_time.png", dpi = 300)
plt.show()

#### Hits associated to correct cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame_time, bins = 300)
plt.title("Correctly associated hits / all hits in frame (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of correctly associated hits")
#plt.savefig("corr_associated_all_30_files_time.png", dpi = 300)

plt.show()

#### Combined histograms

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame_time, bins = 300, label = "Hits correctly associated to cluster / All hits in clusters")
plt.hist(array_frac_uncorr_frame_time, bins = 300, label = "Hits incorrectly associated to cluster / All hits in clusters")
plt.title("Correctly and incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of total hits")
plt.legend()
#plt.savefig("Corr_and_incorr_fractions_all_30_files_time.png", dpi = 300)
plt.show()

### Efficiency vs maximum cluster width

In [ ]:
function_str = "mt_efficiency_as_function_of_cluster_width"
cluster_width_array_used = np.arange(10,101,1)
filename_used = "./testdata/sorted/root_files_fixed/mu3e_sorted_000127.root"
args_time = (filename_used, time_threshold_used, "big", None, None, "time")

#clump.multithreading.run_mt(function_str, src_time, args_time, cluster_width_array = cluster_width_array_used)

In [ ]:
src_cluster_width_time    = src_res + "/efficiency_vs_cluster_width_*_time.txt"
fnames_cluster_width_time = glob(src_cluster_width_time)

array_cluster_width_eff_time  = []
array_cluster_width_time = []
array_cluster_width_tmp_time = []
array_cluster_width_tmp_time.append([np.loadtxt(f) for f in fnames_cluster_width_time])
for i in array_cluster_width_tmp_time[0]:
    array_cluster_width_eff_time.append(float(i[0]))
    array_cluster_width_time.append(float(i[1]))

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(array_cluster_width_time, array_cluster_width_eff_time)
plt.xlabel("Maximum distance between clusters [mm]")
plt.ylabel("Efficiency")
plt.title("Efficiency as function of the maximum distance between clusters with same tid")
#plt.savefig("efficiency_vs_max_distance_threshold_10-100_1_all_frames_each_correct_time.png", dpi=300)
plt.show()

## Pure spatial clustering

In [ ]:
time_threshold_used = 0.4
threshold_cluster_width_used = 999999 

function_str = "mt_compare_to_tid"
src_spatial = "./testdata/sorted/root_files_fixed/mu3e_sorted_000*.root"

args_spatial = (time_threshold_used, threshold_cluster_width_used, "big", None, "iterativespatial")

#clump.multithreading.run_mt(function_str, src_spatial, args_spatial)

In [ ]:
src_frac_corr_frame_spatial          = src_res + "/frac_corr_frame_tid_*_iterativespatial_{}.txt".format(threshold_cluster_width_used)
src_frac_corr_clusters_frame_spatial = src_res + "/frac_corr_clusters_frame_tid_*_iterativespatial_{}.txt".format(threshold_cluster_width_used)
src_frac_uncorr_frame_spatial        = src_res + "/frac_uncorr_frame_tid_*_iterativespatial_{}.txt".format(threshold_cluster_width_used)
src_efficiency_stats_spatial         = src_res + "/efficiency_stats_tid_*_iterativespatial_{}.txt".format(threshold_cluster_width_used)

fnames_frac_corr_frame_spatial           = glob(src_frac_corr_frame_spatial)
fnames_frac_corr_clusters_frame_spatial  = glob(src_frac_corr_clusters_frame_spatial)
fnames_frac_uncorr_frame_spatial         = glob(src_frac_uncorr_frame_spatial)
fnames_efficiency_stats_spatial          = glob(src_efficiency_stats_spatial)

array_frac_corr_frame_spatial          = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_frame_spatial])
array_frac_corr_clusters_frame_spatial = np.concatenate([np.loadtxt(f) for f in fnames_frac_corr_clusters_frame_spatial])
array_frac_uncorr_frame_spatial        = np.concatenate([np.loadtxt(f) for f in fnames_frac_uncorr_frame_spatial])
#array_efficiency_stats         = np.concatenate([np.loadtxt(f) for f in fnames_efficiency_stats])

### Efficiency statistics

In [ ]:
for f in fnames_efficiency_stats_spatial:
    print("File: " + f)
    with open(f, "r", encoding="utf-8") as file:
        for line in file:
            print(line.strip())
    print("\n")

### Efficiency plots

#### Hits associated to wrong cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_uncorr_frame_spatial, bins = 300)
plt.title("Incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of incorrectly associated hits")
#plt.savefig("incorr_associated_all_30_files_spatial.png", dpi = 300)
plt.show()

#### Hits associated to correct cluster

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame_spatial, bins = 300)
plt.title("Correctly associated hits / all hits in frame (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of correctly associated hits")
#plt.savefig("corr_associated_all_30_files_spatial.png", dpi = 300)

plt.show()

#### Combined histograms

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(array_frac_corr_frame_spatial, bins = 300, label = "Hits correctly associated to cluster / All hits in clusters")
plt.hist(array_frac_uncorr_frame_spatial, bins = 300, label = "Hits incorrectly associated to cluster / All hits in clusters")
plt.title("Correctly and incorrectly associated hits / all hits in clusters (per frame)")
plt.xlim(0,1)
plt.xlabel("Fraction of total hits")
plt.legend()
#plt.savefig("Corr_and_incorr_fractions_all_30_files_spatial.png", dpi = 300)
plt.show()

### Efficiency vs maximum cluster width

In [ ]:
function_str = "mt_efficiency_as_function_of_cluster_width"
cluster_width_array_used = np.arange(10,101,1)
filename_used = "./testdata/sorted/root_files_fixed/mu3e_sorted_000127.root"
args_spatial = (filename_used, time_threshold_used, "big", None, None, "iterativespatial")

#clump.multithreading.run_mt(function_str, src_spatial, args_spatial, cluster_width_array = cluster_width_array_used)

In [ ]:
src_cluster_width_spatial    = src_res + "/efficiency_vs_cluster_width_*_iterativespatial.txt"
fnames_cluster_width_spatial = glob(src_cluster_width_spatial)

array_cluster_width_eff_spatial  = []
array_cluster_width_spatial = []
array_cluster_width_tmp_spatial = []
array_cluster_width_tmp_spatial.append([np.loadtxt(f) for f in fnames_cluster_width_spatial])
for i in array_cluster_width_tmp_spatial[0]:
    array_cluster_width_eff_spatial.append(float(i[0]))
    array_cluster_width_spatial.append(float(i[1]))

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.scatter(array_cluster_width_spatial, array_cluster_width_eff_spatial)
plt.xlabel("Maximum distance between clusters [mm]")
plt.ylabel("Efficiency")
plt.title("Efficiency as function of the maximum distance between clusters with same tid")
#plt.savefig("efficiency_vs_max_distance_threshold_10-100_1_all_frames_each_correct.png", dpi=300)
plt.show()